In [3]:
import torch.nn.functional as F
import torch
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from datetime import datetime
import time

In [4]:
BATCH_SIZE = 4
EPOCHS = 2


In [5]:
def train_one_epoch(epoch_index, model, training_loader, info_per_batch, loss_fn, optimizer):
   running_loss = 0.
   last_loss = 0.
   st = time.time()

   # Here, we use enumerate(training_loader) instead of
   # iter(training_loader) so that we can track the batch
   # index and do some intra-epoch reporting
   for i, data in enumerate(training_loader):
      # Every data instance is an input + label pair
      inputs, labels = data

      # Zero your gradients for every batch!
      optimizer.zero_grad()

      # Make predictions for this batch
      outputs = model(inputs)

      # Compute the loss and its gradients
      loss = loss_fn(outputs, labels)
      loss.backward()

      # Adjust learning weights
      optimizer.step()

      # Gather data and report
      running_loss += loss.item()
      if i % 1000 == 999:
         end = time.time()
         last_loss = running_loss / 1000  # loss per batch
         last_time = (end - st) / 1000  # time per batch
         print('batch {} loss: {}'.format(i + 1, last_loss))
         running_loss = 0.
         
         _info_per_batch = {}
         _info_per_batch['epoch'] = epoch_index
         _info_per_batch['batch'] = i + 1
         _info_per_batch['mean_loss'] = last_loss
         _info_per_batch['mean_time'] = last_time
         _info_per_batch['time_image_batch'] = last_time / BATCH_SIZE
         info_per_batch.append(_info_per_batch)

         st = time.time()
         

   return last_loss

In [6]:
def train_epochs(model, training_loader, info_per_batch, loss_fn, optimizer, validation_loader):
   epoch_number = 0

   best_vloss = 1_000_000.

   for epoch in range(EPOCHS):
      print('EPOCH {}:'.format(epoch_number + 1))

      # Make sure gradient tracking is on, and do a pass over the data
      model.train(True)
      avg_loss = train_one_epoch(epoch_number + 1, model, training_loader, info_per_batch, loss_fn, optimizer)

      # We don't need gradients on to do reporting
      model.train(False)

      running_vloss = 0.0
      for i, vdata in enumerate(validation_loader):
         vinputs, vlabels = vdata
         voutputs = model(vinputs)
         vloss = loss_fn(voutputs, vlabels)
         running_vloss += vloss

      avg_vloss = running_vloss / (i + 1)
      print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))


      # Salva o melhor modelo
      # if avg_vloss < best_vloss:
      #    best_vloss = avg_vloss
      #    model_path = 'model_{}_{}'.format(timestamp, epoch_number)
      #    torch.save(model.state_dict(), model_path)

# LeNet

In [7]:
class LeNet(nn.Module):

   def __init__(self):
      super(LeNet, self).__init__()
      # Definição de cada camada da rede neural como atributo da classe
      self.conv1 = nn.Conv2d(1, 6, 5)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(6, 16, 5)
      self.fc1 = nn.Linear(16 * 4 * 4, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

   def forward(self, x):
      # Computação dos passos de uma rede neural até a saída produzida
      x = self.pool(F.relu(self.conv1(x)))
      x = self.pool(F.relu(self.conv2(x)))
      x = x.view(-1, 16 * 4 * 4)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.fc3(x)
      return x

   def num_flat_features(self, x):
      size = x.size()[1:]
      return np.prod(size)
   
model = LeNet()

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [9]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

training_set = torchvision.datasets.FashionMNIST(
    './data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST(
    './data', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(
    training_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(
    validation_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

Training set has 60000 instances
Validation set has 10000 instances


In [10]:
info_per_batch = []

In [11]:
train_epochs(model, training_loader, info_per_batch, loss_fn, optimizer, validation_loader)

EPOCH 1:
batch 1000 loss: 1.9247333088442684
batch 2000 loss: 0.8553839588742703
batch 3000 loss: 0.7070250067040325
batch 4000 loss: 0.6494240892494563
batch 5000 loss: 0.6108041729792021
batch 6000 loss: 0.549041643061908
batch 7000 loss: 0.5509025329215219
batch 8000 loss: 0.5206029276913032
batch 9000 loss: 0.49983676334423943
batch 10000 loss: 0.47071835911343807
batch 11000 loss: 0.4580784262334928
batch 12000 loss: 0.4284323982937494
batch 13000 loss: 0.4293992165924865
batch 14000 loss: 0.4209640056388453
batch 15000 loss: 0.41167909247224455
LOSS train 0.41167909247224455 valid 0.4281473159790039
EPOCH 1:
batch 1000 loss: 0.39714858395492775
batch 2000 loss: 0.3802399608445121
batch 3000 loss: 0.4071223537927726
batch 4000 loss: 0.3690648090149043
batch 5000 loss: 0.3687860967501765
batch 6000 loss: 0.3920790032781661
batch 7000 loss: 0.36273490110854617
batch 8000 loss: 0.3779130775448866
batch 9000 loss: 0.4044479777190136
batch 10000 loss: 0.34476273694064
batch 11000 loss:

# ConvNet

In [12]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
    
model = ConvNet()

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

training_set = torchvision.datasets.MNIST(
    './data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.MNIST(
    './data', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(
    training_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(
    validation_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Training set has 60000 instances
Validation set has 10000 instances


In [15]:
info_per_batch = []

In [16]:
train_epochs(model, training_loader, info_per_batch, loss_fn, optimizer, validation_loader)

EPOCH 1:
batch 1000 loss: 0.915266542147845
batch 2000 loss: 0.32670243506785485
batch 3000 loss: 0.30372809440642595
batch 4000 loss: 0.246821152283228
batch 5000 loss: 0.22014371296297758
batch 6000 loss: 0.20948628368793287
batch 7000 loss: 0.22012425211054507
batch 8000 loss: 0.17692193558806502
batch 9000 loss: 0.1946313565367018
batch 10000 loss: 0.18857012452167693
batch 11000 loss: 0.17499071596813157
batch 12000 loss: 0.17122693366328895
batch 13000 loss: 0.183746412788023
batch 14000 loss: 0.15741507074429684
batch 15000 loss: 0.1538833634554976
LOSS train 0.1538833634554976 valid 0.14506042003631592
EPOCH 1:
batch 1000 loss: 0.14829538930589478
batch 2000 loss: 0.16545096516517516
batch 3000 loss: 0.1331894893800636
batch 4000 loss: 0.16671721210842952
batch 5000 loss: 0.16274361704062176
batch 6000 loss: 0.1515509745500749
batch 7000 loss: 0.1369479431087384
batch 8000 loss: 0.13778779771183328
batch 9000 loss: 0.14997765749251993
batch 10000 loss: 0.13678151208540293
batch

# CustomCNN

In [17]:
class CustomCNN(nn.Module):
   def __init__(self):
      super(CustomCNN, self).__init__()
      # 2 camada convolucionais, 1 camada de pooing e 1 camada linear
      self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5))
      self.batchN1 = nn.BatchNorm2d(num_features=6)
      self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=(5,5))
      self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
      self.batchN2 = nn.BatchNorm1d(num_features=120)
      self.fc2 = nn.Linear(in_features=120, out_features=60)
      self.out = nn.Linear(in_features=60, out_features=10)

   def forward(self, x):
      # hidden conv layer 
      x = self.conv1(x)
      x = F.max_pool2d(input=x, kernel_size=2, stride=2)
      x = F.relu(x)
      x = self.batchN1(x)
      
      # hidden conv layer
      x = self.conv2(x)
      x = F.max_pool2d(input=x, kernel_size=2, stride=2)
      x = F.relu(x)
      
      # flatten
      x = x.reshape(-1, 12*4*4)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.batchN2(x)
      x = self.fc2(x)
      x = F.relu(x)
      
      # output
      x = self.out(x)
      
      return x
   
model = CustomCNN()

In [18]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [19]:
transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

training_set = torchvision.datasets.MNIST(
    './data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.MNIST(
    './data', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(
    training_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(
    validation_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

Training set has 60000 instances
Validation set has 10000 instances


In [20]:
info_per_batch = []

In [21]:
train_epochs(model, training_loader, info_per_batch, loss_fn, optimizer, validation_loader)

EPOCH 1:
batch 1000 loss: 1.0825842409096658
batch 2000 loss: 0.4761379439672455
batch 3000 loss: 0.4385914647215977
batch 4000 loss: 0.3493854379858822
batch 5000 loss: 0.2890595387509093
batch 6000 loss: 0.33691676146443933
batch 7000 loss: 0.2976871873548953
batch 8000 loss: 0.2882055274118902
batch 9000 loss: 0.3216075420464622
batch 10000 loss: 0.26523406780627556
batch 11000 loss: 0.271908985201444
batch 12000 loss: 0.24736220272514037
batch 13000 loss: 0.2523748410275439
batch 14000 loss: 0.22459345625020796
batch 15000 loss: 0.2487560393444437
LOSS train 0.2487560393444437 valid 0.08176848292350769
EPOCH 1:
batch 1000 loss: 0.2439960263131943
batch 2000 loss: 0.24325198863062542
batch 3000 loss: 0.2378574885253329
batch 4000 loss: 0.2099797116574482
batch 5000 loss: 0.2248728931088408
batch 6000 loss: 0.21331114551139763
batch 7000 loss: 0.4714641777734505
batch 8000 loss: 0.351691224899143
batch 9000 loss: 0.3083083811714314
batch 10000 loss: 0.2742406782631879
batch 11000 los